In [1]:
#!pip install scikit-learn
#!pip install opencv-python

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
# Define the directory path containing the images
# batch size and image size are set randomly
data_directory = "/Users/lukasiwan/NeueFische/Repositories/Hydroponics/data/train_data"
batch_size = 32
image_size = (200,200)

In [4]:
# Load and preprocess the data using tf.keras.preprocessing.image_dataset_from_directory
data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training"
)

Found 228 files belonging to 4 classes.
Using 183 files for training.


In [5]:
# Print the class names
class_names = data.class_names
print("Class names:", class_names)

Class names: ['deficiency_nitrogen', 'deficiency_phosphorus', 'deficiency_potassium', 'healthy']


In [6]:
# Convert labels to NumPy arrays
X_train = np.concatenate([x for x, _ in data], axis=0)
y_train = np.concatenate([y for _, y in data], axis=0)


2023-07-14 11:51:42.242336: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-07-14 11:51:42.386547: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-07-14 11:51:42.639833: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-07-14 11:51:43.595458: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-07-14 11:51:43.715526: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-07-14 11:51:43.925490: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


In [7]:
# Convert labels to one-hot encoded format
y_train = to_categorical(y_train, num_classes=4)

In [8]:
# Define the baseline model architecture
model = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

In [9]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 23, 23, 128)       0

In [11]:
# Train the model
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
6/6 [==============================] - 3s 398ms/step - loss: 471.0291 - accuracy: 0.2623
Epoch 2/10
6/6 [==============================] - 2s 386ms/step - loss: 4.4736 - accuracy: 0.3115
Epoch 3/10
6/6 [==============================] - 2s 393ms/step - loss: 1.0622 - accuracy: 0.5137
Epoch 4/10
6/6 [==============================] - 2s 383ms/step - loss: 0.5999 - accuracy: 0.7760
Epoch 5/10
6/6 [==============================] - 2s 386ms/step - loss: 0.2632 - accuracy: 0.8962
Epoch 6/10
6/6 [==============================] - 2s 388ms/step - loss: 0.1879 - accuracy: 0.9617
Epoch 7/10
6/6 [==============================] - 2s 401ms/step - loss: 0.1204 - accuracy: 0.9617
Epoch 8/10
6/6 [==============================] - 2s 405ms/step - loss: 0.3185 - accuracy: 0.9891
Epoch 9/10
6/6 [==============================] - 3s 420ms/step - loss: 0.1489 - accuracy: 0.9563
Epoch 10/10
6/6 [==============================] - 3s 446ms/step - loss: 0.1128 - accuracy: 0.9672


In [12]:
val_data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation"
)

Found 228 files belonging to 4 classes.
Using 45 files for validation.


In [13]:
# Convert labels to NumPy arrays
X_val = np.concatenate([x for x, _ in val_data], axis=0)
y_val = np.concatenate([y for _, y in val_data], axis=0)

2023-07-14 11:52:09.871954: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-07-14 11:52:09.890525: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-07-14 11:52:10.078049: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2023-07-14 11:52:10.090924: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


In [14]:
# Convert labels to one-hot encoded format
y_val = to_categorical(y_val, num_classes=4)

In [15]:
# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

2/2 [==============================] - 0s 59ms/step - loss: 3.3633 - accuracy: 0.3556
Validation Loss: 3.3633272647857666
Validation Accuracy: 0.35555556416511536
